<a href="https://www.kaggle.com/code/aisuko/similar-questions-retrieval?scriptVersionId=162131816" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Overview

In this notebook, we use the semantic search from sentence_transformers to retrieves the most similar questions from the [Quora duplicate questions](https://www.quora.com/q/quoradata/First-Quora-Dataset-Release-Question-Pairs) dataset. And we are going to use `distilbert-multilingual-nil-stsb-quora-ranking`, which was trained to identify similar questions and supports 50+ languages. This is a **symmetric search task**, as the search queries have the same length and content as the questions in the corpus.

In [1]:
!pip install sentence-transformers==2.3.1
!pip install datasets==2.15.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.12.2
    Uninstalling fsspec-2023.12.2:
      Successfully uninstalled fsspec-2023.12.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.1.0
    Uninstalling datasets-2.1.0:
      Successfully uninstalled datasets-2.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf

In [2]:
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login
import os

user_secrets = UserSecretsClient()
login(token=user_secrets.get_secret("HUGGINGFACE_TOKEN"))
os.environ['MODEL_NAME']='quora-distilbert-multilingual'

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Loading and preprocess the data

You can downoad it directly from the internet or use the dataset.

In [3]:
from datasets import load_dataset

max_corpus_size=100000

ds=load_dataset('aisuko/quora_duplicate_questions')
ds

##  You can also download it from the internet.

# from sentence_transformers.util import http_get
# url = "http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv"
# dataset_path = "quora_duplicate_questions.tsv"
# http_get(url,dataset_path)

# # Get all unique sentences from the file
# corpus_sentences = set()
# with open(dataset_path, encoding='utf8') as fIn:
#     reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_MINIMAL)
#     for row in reader:
#         corpus_sentences.add(row['question1'])
#         if len(corpus_sentences) >= max_corpus_size:
#             break

#         corpus_sentences.add(row['question2'])
#         if len(corpus_sentences) >= max_corpus_size:
#             break

# corpus_sentences = list(corpus_sentences)
# print("Encode the corpus. This might take a while")
# corpus_embeddings = model.encode(corpus_sentences, show_progress_bar=True, convert_to_tensor=True)

# ###############################
# print("Corpus loaded with {} sentences / embeddings".format(len(corpus_sentences)))

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

/opt/conda/lib/python3.10/site-packages/datasets/download/streaming_download_manager.py:765: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)


DatasetDict({
    train: Dataset({
        features: ['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate'],
        num_rows: 404290
    })
})

In [4]:
ds=ds['train'].remove_columns(['id','qid1','qid2','is_duplicate'])
ds

Dataset({
    features: ['question1', 'question2'],
    num_rows: 404290
})

In [5]:
# Get all unique sentences from the file
corpus_sentences = set()

num=len(ds['question1'])
num2=len(ds['question2'])

while len(corpus_sentences)<max_corpus_size:
    if num>0:
        for i in ds['question1']:
            corpus_sentences.add(i)
            num-=1
    elif num2>0:
        for i in ds['question2']:
            corpus_sentences.add(i)
            num2-=1
    break

corpus_sentences=list(corpus_sentences)
len(corpus_sentences)

290457

# Loading the model

In [6]:
from sentence_transformers import SentenceTransformer

model=SentenceTransformer(os.getenv('MODEL_NAME'))
model

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/572 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
)

# Computing the embeddings

In [7]:
from sentence_transformers.util import normalize_embeddings

corpus_embeddings=model.encode(corpus_sentences, show_progress_bar=True,convert_to_tensor=True).to('cuda')
corpus_embeddings=normalize_embeddings(corpus_embeddings)
corpus_embeddings

Batches:   0%|          | 0/9077 [00:00<?, ?it/s]

tensor([[ 0.0345,  0.0217,  0.0152,  ...,  0.0271,  0.0134,  0.0015],
        [ 0.0055,  0.0243,  0.0288,  ..., -0.0161,  0.0198, -0.0196],
        [-0.0046,  0.0304,  0.0064,  ..., -0.0253,  0.0138, -0.0212],
        ...,
        [ 0.0254,  0.0198,  0.0009,  ..., -0.0081,  0.0531,  0.0132],
        [ 0.0196,  0.0146,  0.0290,  ..., -0.0158,  0.0581, -0.0084],
        [ 0.0102,  0.0161,  0.0061,  ..., -0.0217, -0.0055, -0.0159]],
       device='cuda:0')

The `dot_score` function implements the `dot product` or `scalar product`. Dot products are very geometric objects. They actually encode relative information about vectors, specifically they tell us "how much" one vector is in the direction of another. https://math.stackexchange.com/a/629350

See how to use it in notebook [Semantic search]([](https://www.kaggle.com/code/aisuko/semantic-search) )

In [8]:
import time
from sentence_transformers.util import semantic_search

# Function that searches the corpus and prints the results
def search(inp_question, top_k):
    start_time=time.time()
    question_embedding=model.encode(inp_question,show_progress_bar=True, convert_to_tensor=True).to('cuda')
#     question_embedding=normalize_embeddings(question_embedding)
    hits=semantic_search(
        question_embedding, 
        corpus_embeddings,
        query_chunk_size=100,
        corpus_chunk_size=500000,
        top_k=top_k
#         score_function=dot_score
    )
    
    end_time=time.time()
    # only on the first element
    hits=hits[0]
    
    print('Input question',{inp_question})
    print('Results (after {:.3f} seconds):'.format(end_time-start_time))
    
    for hit in hits:
        print("\t{:.3f}\t{}".format(hit['score'], corpus_sentences[hit['corpus_id']]))

In [9]:
top_k=5

search('How can I learn Python online?', top_k)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Input question {'How can I learn Python online?'}
Results (after 0.048 seconds):
	0.988	How and from where can I learn Python language via video tutorials online?
	0.983	Which is the best resource to learn Python? Online, books, video tutes?
	0.982	What is the best online resource to learn Python?
	0.981	How do I learn Python at home?
	0.980	Where I should learn Python?


In [10]:
# German: How can I learn Python online?
search("Wie kann ich Python online lernen?", top_k)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Input question {'Wie kann ich Python online lernen?'}
Results (after 0.030 seconds):
	0.988	How and from where can I learn Python language via video tutorials online?
	0.985	Which is the best resource to learn Python? Online, books, video tutes?
	0.984	What is the best online resource to learn Python?
	0.981	How do I learn Python at home?
	0.980	Where I should learn Python?
